In [15]:
import ollama
import glob
from PIL import Image

In [16]:
LIMIT = 5 

In [18]:
all_images = glob.glob("../data/coco/val2017/*.jpg")

images_to_process = ["../data/coco/val2017/000000179765.jpg"]

print(f"--- Démarrage de l'analyse ({len(images_to_process)} images sélectionnées sur {len(all_images)}) ---")

for img_path in images_to_process:
    print(f"Analyse de : {img_path}")
    
    try:
        reponse = ollama.chat(
            model='llava',
            messages=[{
                'role': 'user',
                'content': 'Make a caption describing the image.',
                'images': [img_path]
            }]
        )
        
        description = reponse['message']['content']
        print(f"Légende générée : {description}\n")
        
    except Exception as e:
        print(f"Erreur sur l'image {img_path}: {e}")

img = Image.open("../data/coco/val2017/000000179765.jpg")
img.show()

print("--- Fin du traitement ---")

--- Démarrage de l'analyse (1 images sélectionnées sur 5000) ---
Analyse de : ../data/coco/val2017/000000179765.jpg
Légende générée :  "Classic Rebel, Classic Ride: A vintage motorcycle stands as timeless as its design." 

--- Fin du traitement ---
